In [1]:
from utils.get_data import generate_data_nested_dirs, get_cube_at_point, generate_cubes, generate_cube_batch
import numpy as np

In [2]:
import pandas as pd

df_annotations = pd.read_csv('CSVFILES/annotations.csv')

In [3]:
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dense, Dropout, ReLU

In [4]:
model = Sequential()
model.add(Conv3D(
        filters = 64,
        kernel_size = (15,15,15),
        strides = (1,1,1),
        padding = 'valid',
        activation = 'relu',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None,
        input_shape = (25,64,64,1)))
model.add(MaxPooling3D(
        pool_size = (2,2,2),
        strides = None,
        padding = 'valid'))
model.add(BatchNormalization(axis = 1))
model.add(Dropout(0.2))
model.add(Flatten(input_shape=(25,64,64,1)))
# model.add(Dense(
#         units = 256,
#         activation = 'relu',
#         use_bias = True,
#         kernel_initializer = 'glorot_uniform',
#         bias_initializer = 'zeros',
#         kernel_regularizer = None,
#         bias_regularizer = None,
#         activity_regularizer = None,
#         kernel_constraint = None,
#         bias_constraint = None))
model.add(Dense(
        units = 32,
        activation = 'relu',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.add(Dense(
        units = 2,
        activation = 'softmax',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 11, 50, 50, 64)    216064    
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 25, 25, 64)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 25, 25, 64)     20        
_________________________________________________________________
dropout (Dropout)            (None, 5, 25, 25, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 200000)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                6400032   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6

In [5]:
sgd = optimizers.SGD(
    lr=0.01, 
    decay = 1e-3, 
    nesterov=False
)

model.compile(
    optimizer=sgd,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
train_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 16,
    rotate=30,
    axes=(0,1,2),
    offset=0.25
)

test_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/test',
    batch_size = 16
)

val_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/val',
    batch_size = 16
)

model.fit(
    x = train_generator,
    validation_data = val_generator,
    validation_steps = 25,
    epochs = 2,
    shuffle=True,
    steps_per_epoch = 75
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 75 steps, validate for 25 steps
Epoch 1/2
75/75 [==============================] - 1776s 24s/step - loss: 2.9713 - accuracy: 0.6683 - val_loss: 0.6857 - val_accuracy: 0.5225
Epoch 2/2
75/75 [==============================] - 1810s 24s/step - loss: 0.4789 - accuracy: 0.8342 - val_loss: 0.6596 - val_accuracy: 0.6000


In [ ]:
model.save('first_model.h5')

In [ ]:
y_pred = model.predict(train_generator, steps = 25)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
test_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 16
)

y = np.zeros((1,2))
for i in range(25):
    x, batch = next(test_generator)
    if not y.any():
        y = np.argmax(batch, axis=1)
        continue
    y = np.concatenate((y, np.argmax(batch, axis=1)))

In [ ]:
y_pred

In [ ]:
confusion_matrix(y,y_pred)